In [ ]:
!pip install unstructured
!pip install sentence-transformers
!pip install chromadb
!pip install openai
!pip install langchain-openai

In [ ]:
# 데이터 불러오기
from langchain.document_loaders import TextLoader
documents = TextLoader("./datos/España.txt").load()

# 데이터의 문장을 청크로 분할. (큰 덩어리의 문서를 작은 덩어리로 분할)
from langchain.text_splitter import RecursiveCharacterTextSplitter

# (1) 문서를 청크로 분할
def split_docs(documents, chunk_size = 1000, chunk_overlap = 20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                                   chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

# (2) docs 변수에 분할 문서를 저장
docs = split_docs(documents)

# 벡터 데이터베이스인 크로마에 임베딩 처리된 벡터를 저장. 임베딩 처리는 open ai 모델을 사용
from langchain_openai import OpenAIEmbeddings
api_key = "write your api key"
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002", api_key = api_key)

# (1) Chromdb에 벡터 저장
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings, persist_directory = "./data")

# 텍스트 파일에서 관련 내용을 찾아 LLM에 제공하면 LLM이 답변을 생성
from langchain.chat_models import ChatOpenAI
model_name = "gpt-4-turbo"
llm = ChatOpenAI(model_name=model_name, api_key = api_key)

#Q&A 체인을 사용하여 쿼리에 대한 답변 얻기
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff", verbose = True)

#쿼리를 작성하고 유사도 검색을 수행하여 답변을 생성, 따라서 텍스트에 있는 내용을 물어봐야 함
query = "¿Dónde está España?"
matching_docs = db.similarity_search(query)
answer = chain.run(input_documents = matching_docs, question = query)
answer